In [70]:
from duckduckgo_search import DDGS
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
chato=DDGS()

## CHAT-AI Duck Duck GO

In [1]:
import requests
import json

class ChatSession:
    def __init__(self):
        self.client = requests.Session() 
        self._chat_vqd = None
        self._chat_messages = []
        self._chat_tokens_count = 0

    def chat(self, keywords: str, model: str = "gpt-4o-mini", timeout: int = 30) -> str:
        models_deprecated = {
            "gpt-3.5": "gpt-4o-mini",
            "llama-3-70b": "llama-3.1-70b",
        }
        if model in models_deprecated:
            print(f"{model=} is deprecated, using {models_deprecated[model]}")
            model = models_deprecated[model]
        
        models = {
            "claude-3-haiku": "claude-3-haiku-20240307",
            "gpt-4o-mini": "gpt-4o-mini",
            "llama-3.1-70b": "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            "mixtral-8x7b": "mistralai/Mixtral-8x7B-Instruct-v0.1",
        }

        # Get vqd if not already set
        if not self._chat_vqd:
            resp = self.client.get("https://duckduckgo.com/duckchat/v1/status", headers={"x-vqd-accept": "1"})
            self._chat_vqd = resp.headers.get("x-vqd-4", "")

        # Add user message to chat history
        self._chat_messages.append({"role": "user", "content": keywords})
        self._chat_tokens_count += len(keywords) // 4 if len(keywords) >= 4 else 1  # approximate number of tokens

        json_data = {
            "model": models[model],
            "messages": self._chat_messages,
        }

        # Send request to DuckDuckGo AI
        resp = self.client.post(
            "https://duckduckgo.com/duckchat/v1/chat",
            headers={"x-vqd-4": self._chat_vqd},
            json=json_data,
            timeout=timeout,
        )
        self._chat_vqd = resp.headers.get("x-vqd-4", "")
        
        # Process the response
        data = ",".join(x for line in resp.text.rstrip("[DONE]LIMT_CVRSA\n").split("data:") if (x := line.strip()))
        data = json.loads("[" + data + "]")

        results = []
        for x in data:
            if x.get("action") == "error":
                err_message = x.get("type", "")
                if x.get("status") == 429:
                    raise (
                        Exception("ConversationLimitException: " + err_message)
                        if err_message == "ERR_CONVERSATION_LIMIT"
                        else Exception("RatelimitException: " + err_message)
                    )
                raise Exception("DuckDuckGoSearchException: " + err_message)
            elif message := x.get("message"):
                results.append(message)
        result = "".join(results)

        # Add AI response to chat history
        self._chat_messages.append({"role": "assistant", "content": result})
        self._chat_tokens_count += len(results)
        return result



In [18]:
chat_session = ChatSession()

questions = [
    "Hi", "My name is Faijan", "What is my name?", "How are you doing today?",
    "What were we talking about?", "Do you remember my name?",
    "Can you tell me a joke?", "What's your favorite color?",
    "Do you have any hobbies?", "If you could travel anywhere, where would you go?", 
    "What's the weather like in London?", "I'm feeling sad. Can you help?",
    "Explain quantum physics to a 5-year-old.",
    "I need to schedule a meeting for next Tuesday.",
    "What are the benefits of exercise?", "Tell me about the history of the internet.",
    "What are some good books to read?", "What's the meaning of life?", 
    "Translate 'hello' into Spanish.", "Convert 10 kilometers into miles.",
    "If all cats are mammals, and all mammals are warm-blooded, are all cats warm-blooded?",
    "You are in a room with two doors. One leads to freedom, and the other to a lion. There are two guards, one always tells the truth, and the other always lies. You can ask one guard one question. What do you ask to find the door to freedom?",
    "What is the next number in this sequence: 2, 4, 6, 8...?", 
    "A bat and a ball cost $1.10. The bat costs $1.00 more than the ball. How much does the ball cost?",
    "Write a short story about a talking cat.", "Compose a poem about the ocean.",
    "If animals could talk, which would be the most interesting to have a conversation with?",
    "Imagine you could time travel - where would you go?",
    "What is the sound of one hand clapping?",  "What happens when you divide by zero?",
    "This is a very, very, very long sentence that just keeps going on and on and on and on...", 
    "#$%^&*",  "Can you show me an image of a [nonexistent thing]?", 
    "How can I get help using this chatbot?", "Can you repeat what you just said?",
    "Are you a real person?", "What are your limitations?",
    "Can you learn from our conversations?", "Is there anything else you can do?",
    "What is your opinion on the use of artificial intelligence in art?"
]

for q in questions:
    print(f"Question: {q}")
    print(f"Answer: {chat_session.chat(q)}")
    print()

Question: Hi
Answer: Hello! How can I assist you today?

Question: My name is Faijan


Exception: RatelimitException: ERR_SERVICE_UNAVAILABLE

In [2]:

chat_session = ChatSession()
question=["hi","my name is faijan","what is my name"]

for q in question:
    print(f"Question: {q}")
    print(f"Answer: {chat_session.chat(q)}")
    print()


Question: hi
Answer: Hello! How can I assist you today?

Question: my name is faijan


Exception: RatelimitException: ERR_SERVICE_UNAVAILABLE

In [3]:
from duckduckgo_search import DDGS



In [4]:
chato=DDGS()

In [8]:
chato.chat("my name is faijan and i am a programmer")

'Nice to meet you, Faijan! As a programmer, what languages or technologies do you work with?'

In [9]:
chato.chat("what is my name?")

'Your name is Faijan.'

## News

In [5]:
chato.news("what is bitcoin price todays?",max_results=6)

[{'date': '2024-08-20T12:36:00+00:00',
  'title': 'Bitcoin price today: BTC is trading at $60,547.22',
  'body': "Bitcoin, or 1 BTC, traded at $60,547.22 as of 8 a.m. ET. The highest intraday price the crypto reached in the past year was $73,750.07 on March 14, 2024. The above chart pulls data as of 8 a.m. ET daily. It doesn't display intraday highs or lows.",
  'url': 'https://www.usatoday.com/money/blueprint/investing/cryptocurrency/bitcoin-price-today-08-20-2024/',
  'image': 'https://www.usatoday.com/money/blueprint/images/uploads/2024/01/29123416/Bitcoin-scaled-e1706546105671.jpg',
  'source': 'USA Today'},
 {'date': '2024-08-20T14:20:00+00:00',
  'title': 'Bitcoin Price Forecast - Bitcoin Continues to See Sideways Action',
  'body': 'The Bitcoin market continues to see a lot of sideways action, which of course makes a certain amount of sense, as the market is now waiting to see what the Wall Street based ETF will end up doing to the market overall.',
  'url': 'https://www.fxempir

In [6]:
chato.news("what is bitcoin price todays?")

[{'date': '2024-08-20T12:36:00+00:00',
  'title': 'Bitcoin price today: BTC is trading at $60,547.22',
  'body': "Bitcoin, or 1 BTC, traded at $60,547.22 as of 8 a.m. ET. The highest intraday price the crypto reached in the past year was $73,750.07 on March 14, 2024. The above chart pulls data as of 8 a.m. ET daily. It doesn't display intraday highs or lows.",
  'url': 'https://www.usatoday.com/money/blueprint/investing/cryptocurrency/bitcoin-price-today-08-20-2024/',
  'image': 'https://www.usatoday.com/money/blueprint/images/uploads/2024/01/29123416/Bitcoin-scaled-e1706546105671.jpg',
  'source': 'USA Today'},
 {'date': '2024-08-20T14:20:00+00:00',
  'title': 'Bitcoin Price Forecast - Bitcoin Continues to See Sideways Action',
  'body': 'The Bitcoin market continues to see a lot of sideways action, which of course makes a certain amount of sense, as the market is now waiting to see what the Wall Street based ETF will end up doing to the market overall.',
  'url': 'https://www.fxempir

## Images Search

In [7]:
chato.images("cricket t20 world cup 2024",max_results=7)

[{'title': 'Ind vs SA Highlights, T20 World Cup 2024 Final: We Are The Champions ...',
  'image': 'https://c.ndtvimg.com/2024-06/e3d6su2o_india-afp_625x300_30_June_24.jpg?im=FaceCrop,algorithm=dnn,width=1200,height=675',
  'thumbnail': 'https://tse2.mm.bing.net/th?id=OIP.8gRCZCAqdyu5aoRSkgdeoQHaEK&pid=Api',
  'url': 'https://sports.ndtv.com/t20-world-cup-2024/ind-vs-sa-final-live-cricket-score-t20-world-cup-2024-india-vs-south-africa-live-score-and-commentary-29-june-5994843',
  'height': 675,
  'width': 1200,
  'source': 'Bing'},
 {'title': 'India wins the ICC T20 Cricket World Cup 2024. : r/cricketworldcup',
  'image': 'https://preview.redd.it/india-wins-the-icc-t20-cricket-world-cup-2024-v0-gpj4h5h2wj9d1.png?auto=webp&s=06ed7670412d502b6af9f62ae78954e3527aa9e4',
  'thumbnail': 'https://tse2.mm.bing.net/th?id=OIP.A7c3uVU3KeMmve9fL0HT1AAAAA&pid=Api',
  'url': 'https://www.reddit.com/r/cricketworldcup/comments/1drh39b/india_wins_the_icc_t20_cricket_world_cup_2024/',
  'height': 592,
  

## Vedio Search

In [8]:
chato.videos("most gold medal for olyphmic 2024 ",max_results=6)

[{'content': 'https://www.youtube.com/watch?v=uJs693eNfuQ',
  'description': "Re-live ALL the incredible #Paris2024 action: https://oly.ch/ParisReplays Watch the full replay of the Men's Basketball Gold Medal Match between the United States and France from the #Paris2024 Olympic Games. 🥇 Team USA 🇺🇸 🥈 Team France 🇫🇷 🥉 Team Serbia 🇷🇸 _____________________________________________________ 🇨🇳 ...",
  'duration': '2:14:47',
  'embed_html': '<iframe width="1280" height="720" src="https://www.youtube.com/embed/uJs693eNfuQ?autoplay=1" frameborder="0" allowfullscreen></iframe>',
  'embed_url': 'https://www.youtube.com/embed/uJs693eNfuQ?autoplay=1',
  'image_token': 'dac247e1169781afb23028136084344b5256cddbd4f076ceea0449138a9b9dca',
  'images': {'large': 'https://tse2.mm.bing.net/th?id=OVF.vUEwe0q18e5mwrUsed5FKw&pid=Api',
   'medium': 'https://tse2.mm.bing.net/th?id=OVF.vUEwe0q18e5mwrUsed5FKw&pid=Api',
   'motion': '',
   'small': 'https://tse2.mm.bing.net/th?id=OVF.vUEwe0q18e5mwrUsed5FKw&pid=Ap

## Text Search

In [66]:
chato.text("who won highest medal in olyphmic 2024",max_results=5)

[{'title': 'Who won the Olympics? Final medal count, standings, most gold winners ...',
  'href': 'https://www.sportingnews.com/us/olympics/news/olympics-winner-2024-final-medal-standings-usa/10d8066d05bb7d20c197104f',
  'body': 'MORE: The greatest 33 U.S. Summer Olympians of all time Olympic medal leaders by year. The United States has finished atop the medal count in seven of the last 10 Summer Olympics. The last nation ...'},
 {'title': 'Who is winning the Olympics? Updated 2024 medal standings, most gold ...',
  'href': 'https://www.sportingnews.com/us/olympics/news/olympics-winner-2024-medal-standings/56aeaa329e2aa38435ac7b62',
  'body': 'The United States again comfortably finished with the most medals at the 2020 Olympics in Tokyo, winning 113 overall and 39 gold, but the gap narrowed from 2016 when the U.S. won by a margin of 51 ...'},
 {'title': 'List of 2024 Summer Olympics medal winners - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/List_of_2024_Summer_Olympics_medal

In [67]:
chato.text("what is the whether today is Goa India",max_results=5)

[{'title': 'Weather in Goa, India | timeanddate.com',
  'href': 'https://www.timeanddate.com/weather/india/goa',
  'body': 'Current weather in Goa, with temperatures in 3 cities, including high and low temperatures for the country.'},
 {'title': '403516, Goa, India Weather Forecast | AccuWeather',
  'href': 'https://www.accuweather.com/en/in/403516/403516/weather-forecast/528454_pc',
  'body': '403516, Goa, India Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'},
 {'title': 'Goa (India) weather | Met Office',
  'href': 'https://www.metoffice.gov.uk/weather/forecast/tdsxe1jgg',
  'body': 'Coondapur 131 miles. View all locations in India. Goa 7 day weather forecast including weather warnings, temperature, rain, wind, visibility, humidity and UV.'},
 {'title': 'Hourly forecast for Goa, Goa, India | timeanddate.com',
  'href': 'https://www.timeanddate.com/weather/@1271157/hourly',
  'body': 'Sun & Moon. Weather Today Weather Hourly 14 D

In [11]:
import cohere
from dotenv import load_dotenv
import os
load_dotenv()
co = cohere.Client(api_key=os.getenv("COHERENT_API_KEY"))

docs = [
    "Carson City is the capital city of the American state of Nevada.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
    "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
    "Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
]

response = co.rerank(
    model="rerank-english-v3.0",
    query="What is the capital of the United States?",
    documents=docs,
    top_n=3,
)
print(response)


c:\Users\faiza\Music\llmResearch\gemini-api-python-function-calling-sample\env\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


id='900f22d0-a6d0-4bb5-af0b-98557956cfed' results=[RerankResponseResultsItem(document=None, index=3, relevance_score=0.999071), RerankResponseResultsItem(document=None, index=4, relevance_score=0.7867867), RerankResponseResultsItem(document=None, index=0, relevance_score=0.32713068)] meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None)


In [68]:
query="what is the today tempature in Goa Quepem"
search_res=chato.text(query,max_results=6)
search_res

[{'title': 'Quepem, Goa, India Weather Forecast | AccuWeather',
  'href': 'https://www.accuweather.com/en/in/quepem/202400/weather-forecast/202400',
  'body': 'Quepem, Goa, India Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'},
 {'title': 'Quepem, Goa, India Current Weather | AccuWeather',
  'href': 'https://www.accuweather.com/en/in/quepem/202400/current-weather/202400',
  'body': 'Current weather in Quepem, Goa, India. Check current conditions in Quepem, Goa, India with radar, hourly, and more.'},
 {'title': 'Quepem, Goa, India Hourly Weather | AccuWeather',
  'href': 'https://www.accuweather.com/en/in/quepem/202400/hourly-weather-forecast/202400',
  'body': 'Hourly weather forecast in Quepem, Goa, India. Check current conditions in Quepem, Goa, India with radar, hourly, and more.'},
 {'title': 'Hourly forecast for Quepem, Goa, India - timeanddate.com',
  'href': 'https://www.timeanddate.com/weather/@1259098/hourly',
  'body': '

In [71]:
import google.generativeai as genai
load_dotenv()

genai.configure(api_key=os.getenv('GEMINI_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-flash')

print("-" * 100)
print("response")
result = model.generate_content(
    f"""
    Based on this information: `{search_res}` 
    and this question: `{query}`
    respond to the user in a friendly manner.
    """
)
to_markdown(result.text)

----------------------------------------------------------------------------------------------------
response


> It looks like the temperature in Quepem, Goa is currently 77°F!  That's according to timeanddate.com. 
> 
> Do you want to know more about the weather in Quepem, like the forecast for the next few days, or maybe some hourly details? Just ask! 😊 


In [73]:
# rerank
query="what is the today tempature in Goa Quepem"
text_results=chato.text(query,max_results=6)
docs=[search['body'] for search in text_results]
response = co.rerank(
    model="rerank-english-v3.0",
    query=query,
    documents=docs,
    top_n=3,
)

print(response)

id='4d4d38ff-ccfe-415f-8e2d-e4d54fb2f977' results=[RerankResponseResultsItem(document=None, index=5, relevance_score=0.9935369), RerankResponseResultsItem(document=None, index=4, relevance_score=0.9768466), RerankResponseResultsItem(document=None, index=1, relevance_score=0.83169055)] meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None)


In [74]:
docs

['Quepem, Goa, India Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.',
 'Current weather in Quepem, Goa, India. Check current conditions in Quepem, Goa, India with radar, hourly, and more.',
 'Get accurate hourly forecasts for today, tonight, and tomorrow, along with 10-day daily forecasts and weather radar for Quepem, Goa with MSN Weather. Stay updated on precipitation, severe weather ...',
 'Hourly weather forecast in Quepem, Goa, India. Check current conditions in Quepem, Goa, India with radar, hourly, and more.',
 '99 / 79 °F. 10. 98 / 80 °F. 11. 98 / 80 °F. Detailed forecast for 14 days. Need some help? Current weather in Quepem and forecast for today, tomorrow, and next 14 days.',
 'Hour-by-Hour Forecast for Quepem, Goa, India. Time/General. Weather. Time Zone. DST Changes. Sun & Moon. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 77 °F. Light rain.']

In [75]:
response.results[0].index

5

In [76]:

print("-" * 100)
result = model.generate_content(
    f"""
    Based on this information: `{docs}` 
    and this question: `{query}`
    respond to the user in a friendly manner.
    """
)
to_markdown(result.text)

----------------------------------------------------------------------------------------------------


> It looks like the temperature in Quepem, Goa today is 77°F!  It's currently light rain, so you might want to grab an umbrella if you're heading out. 😊 


In [84]:
import cohere
from dotenv import load_dotenv
import os
load_dotenv()
co = cohere.Client(api_key=os.getenv("COHERENT_API_KEY"))

def generate(query:str):
    print("query: " + query)

    # web search duck duck go
    text_results=chato.text(query,max_results=6)
    print("result:\n",text_results)
    docs=[search['body'] for search in text_results]

    # rerank
    response = co.rerank(
        model="rerank-english-v3.0",
        query=query,
        documents=docs,
        top_n=3,
    )

    print("reRank",response)
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("-" * 100)
    print("response :")
    result = model.generate_content(
        f"""
        Based on this information: `{docs}` 
        and this question: `{query}`
        respond to the user in a friendly manner.
        """
    )
    return result.text

query="who won cricket 2024 t20 world cup"
to_markdown(generate(query))

query: who won cricket 2024 t20 world cup
result:
 [{'title': 'India script stunning title win - International Cricket Council', 'href': 'https://www.icc-cricket.com/tournaments/t20cricketworldcup/videos/india-script-stunning-title-win-match-highlights-sa-v-ind-t20wc-2024-final', 'body': "ICC Men's T20 World Cup, 2024. India script stunning title win | Match Highlights | SA v IND | T20WC 2024 Final. 29 June, 2024. Latest Videos. ICC Cricket World Cup, 2023. Power Performances presented by Aramco | Episode 3 | CWC 2023. 20h. Cricket World Cup. Best of KL Rahul | CWC23. 22 December, 2023."}, {'title': "2024 ICC Men's T20 World Cup final - Wikipedia", 'href': "https://en.wikipedia.org/wiki/2024_ICC_Men's_T20_World_Cup_final", 'body': "The 2024 ICC Men's T20 World Cup final was a Twenty20 International cricket match played at Kensington Oval in Bridgetown, Barbados on 29 June 2024 to determine the winner of the 2024 ICC Men's T20 World Cup. [1] [2] It was played between South Africa and In

> Great question!  🎉  India won the 2024 T20 Cricket World Cup!  They beat South Africa in a thrilling final match on June 29th. 🏏🏆 


In [79]:
query="who won highest medal in olymphic 2024"
to_markdown(generate(query))

query: who won highest medal in olymphic 2024
result:
 [{'title': 'Who won the Olympics? Final medal count, standings, most gold winners ...', 'href': 'https://www.sportingnews.com/us/olympics/news/olympics-winner-2024-final-medal-standings-usa/10d8066d05bb7d20c197104f', 'body': 'MORE: The greatest 33 U.S. Summer Olympians of all time Olympic medal leaders by year. The United States has finished atop the medal count in seven of the last 10 Summer Olympics. The last nation ...'}, {'title': 'Medal Count - Paris 2024 Olympic Medal Table', 'href': 'https://olympics.com/en/paris-2024/medals?displayAsWebViewdark=true', 'body': 'Official medal standings for the Paris 2024 Olympics (Jul 26-Aug 11, 2024). Find out which athletes are bringing home medals and breaking records. ... Does not include medals won by AIN. Timing and scoring provided by OMEGA. Results powered by Atos. POWERED BY.'}, {'title': 'Who is winning the Olympics? Updated 2024 medal standings, most gold ...', 'href': 'https://ww

> That's a great question!  Unfortunately, I can't give you a definitive answer about who won the "highest medal" in the 2024 Olympics.  It looks like the information I have focuses on overall medal counts for countries, not individual athletes.  
> 
> It does mention that the U.S. won the most medals overall (126) with 40 gold medals.  Katie Ledecky won two of those gold medals for the U.S.  It also mentions that France won the gold medal in men's Rugby Sevens. 
> 
> To find out who won the highest medal for a specific sport, you'd need to look up the results for that event! 😊 


In [81]:
query="who won men table tennis gold medal in olymphic 2024"
to_markdown(generate(query))

query: who won men table tennis gold medal in olymphic 2024
result:
 [{'title': "Paris 2024 table tennis: All results, as the People's Republic of China ...", 'href': 'https://olympics.com/en/news/paris-2024-table-tennis-all-results-people-s-republic-of-china-win-gold-men-team', 'body': "The People's Republic of China defended their gold medal in the men's table tennis team event in emphatic fashion, beating Sweden 3-0 at South Paris Arena on Friday, 9 August at the Olympic Games Paris 2024. It was their fifth title in the men's team event, a competition they have yet to lose since it debuted at the Olympic Games Beijing ..."}, {'title': "Ma Long vs Fan Zhendong | Men's Singles Table Tennis Gold Medal ...", 'href': 'https://www.youtube.com/watch?v=VTCDQYYKA9o', 'body': "📺 Re-live ALL the incredible #Paris2024 action ️ https://go.olympics.com/watch MA Long wins historic second straight Olympic gold in men's singles table te..."}, {'title': "Paris 2024 Olympics: China's Fan wins table te

> That's a great question! 🎉 
> 
> It looks like **China** won the gold medal in the **men's table tennis team event** at the 2024 Olympics.  They beat Sweden 3-0 in the final match. 
> 
> But if you're asking about the **men's singles** event, the gold medal went to **Fan Zhendong** of China! He beat Truls Moregard of Sweden to claim the victory. 🏆 


In [28]:
query="hey"
to_markdown(generate(query))

query: hey
result:
 [{'title': 'HEY | definition in the Cambridge English Dictionary', 'href': 'https://dictionary.cambridge.org/us/dictionary/english/hey', 'body': "HEY meaning: 1. used as a way of attracting someone's attention, sometimes in a way that is not very polite: 2…. Learn more."}, {'title': 'Hey - Red Hot Chili Peppers (lyrics on screen) - YouTube', 'href': 'https://www.youtube.com/watch?v=m1xN7w07EWM', 'body': 'hey by the red hot chili peppers. all of this is owned by the red hot chili peppers and warner brothers. no copyright infringement intended.enjoy(:'}, {'title': 'HEY — A delightfully fresh take on email + calendar, from 37signals', 'href': 'https://www.hey.com/', 'body': "HEY is a new email and calendar app that lets you control who can email you, organize your messages in different boxes, and browse newsletters and long-reads. It's a fresh take on email by the creators of Basecamp and 37signals."}, {'title': 'Hey Definition & Meaning - Merriam-Webster', 'href': 'ht

'Hey there! 👋  You\'re asking about the word "hey."  It\'s a pretty versatile word, huh? It can be used in a lot of different ways.  Do you want to know more about its different meanings, how people use it, or maybe some examples?  Just let me know! 😊 \n'

## Query Classification

In [96]:
from dotenv import load_dotenv
import os
import cohere
import json
# Load environment variables
load_dotenv()

# co = cohere.Client(api_key=os.getenv("COHERENT_API_KEY"))

def websearch(query: str):
    print("Query:", query)

    # Web search using DuckDuckGo (assuming chato.text is a function for this)
    text_results = chato.text(query, max_results=6)
    print("Results:\n", text_results)
    docs = [search['body'] for search in text_results]

    # Rerank results
    rerank_response = co.rerank(
        model="rerank-english-v3.0",
        query=query,
        documents=docs,
        top_n=3
    )

    print("Reranked:", rerank_response)

    # Use the generative model for content generation
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("-" * 100)
    print("Response:")
    result = model.generate_content(
        f"""
        Based on this information: `{docs}` 
        and this question: `{query}`,
        respond to the user in a friendly manner.
        """
    )
    return result.text

def classify_query(user_query: str):

    # System prompt to classify the query
    system_prompt = f"""
    You are an AI classifier. Determine if the following question requires real-time or up-to-date knowledge to answer. Respond in JSON format as {{"results": "yes"}} if it requires real-time data, or {{"results": "no"}} if it does not. Do not provide any explanation.

    Question: "{user_query}"
    """

    # Generate classification using the generative model
    model = genai.GenerativeModel('gemini-1.5-flash')
    classification_result = json.loads(model.generate_content(system_prompt).text)
    print(classification_result)
    if classification_result['results'] == "yes":
        # Websearch to get up-to-date information
        return websearch(user_query)
    else:
        # Normal LLM query response
        return model.generate_content(user_query).text


# Example usage
result = classify_query("who won cricket t20 2024")
print(result)


{'results': 'yes'}
Query: who won cricket t20 2024
Results:
 [{'title': "2024 ICC Men's T20 World Cup final - Wikipedia", 'href': 'https://en.wikipedia.org/wiki/2024_Cricket_World_Cup_final', 'body': "The 2024 ICC Men's T20 World Cup final was a Twenty20 International cricket match played at Kensington Oval in Bridgetown, Barbados on 29 June 2024 to determine the winner of the 2024 ICC Men's T20 World Cup. [1] [2] It was played between South Africa and India.[3]India defeated South Africa by 7 runs to win their second T20 World Cup title. [4] Virat Kohli was named Player of the Match for ..."}, {'title': 'India script stunning title win - International Cricket Council', 'href': 'https://www.icc-cricket.com/tournaments/t20cricketworldcup/videos/india-script-stunning-title-win-match-highlights-sa-v-ind-t20wc-2024-final', 'body': "ICC Men's T20 World Cup, 2024. India script stunning title win | Match Highlights | SA v IND | T20WC 2024 Final. 29 June, 2024. Latest Videos. ICC Cricket World

In [97]:
# Example usage
result = classify_query("hello")
print(result)


{'results': 'no'}
Hello! How can I help you today? 



In [57]:
# Example usage
result = classify_query("how are you")
print(result)


----------------------------------------------------------------------------------------------------
Response:
{'results': 'no'}
I am an AI language model, so I don't have feelings like humans do. However, I am here to assist you with any questions or tasks you may have. How can I help you today? 



In [94]:
import json


# Example usage
result = classify_query("Who won the cricket T20 in 2024?")
print(result)


{'results': 'yes'}
Query: Who won the cricket T20 in 2024?
Results:
 [{'title': "2024 ICC Men's T20 World Cup final - Wikipedia", 'href': 'https://en.wikipedia.org/wiki/2024_Cricket_World_Cup_final', 'body': "The 2024 ICC Men's T20 World Cup final was a Twenty20 International cricket match played at Kensington Oval in Bridgetown, Barbados on 29 June 2024 to determine the winner of the 2024 ICC Men's T20 World Cup. [ 1][ 2] It was played between South Africa and India. [ 3]"}, {'title': 'India script stunning title win | Match Highlights | SA v IND | T20WC ...', 'href': 'https://www.icc-cricket.com/tournaments/t20cricketworldcup/videos/india-script-stunning-title-win-match-highlights-sa-v-ind-t20wc-2024-final', 'body': "India script stunning title win | Match Highlights | SA v IND | T20WC 2024 Final. overview U19 Cricket World Cup Highlighted Videos World Test Championship. ICC Men's T20 World Cup, 2024."}, {'title': "India wins men's T20 World Cup, defeating South Africa in ... - CNN",

In [95]:
result = classify_query("Who won the cricket T20 world in 2014?")
print(result)


{'results': 'no'}
**Sri Lanka** won the Cricket T20 World Cup in 2014. 



In [ ]:
result = classify_query("Who won the cricket T20 world in 2014?")
print(result)
